## Read and Split Data

In [12]:
# Read data here
# Extract zip file if folder does not already exist
import os
import zipfile
import json

folder_path = 'data/CT23_1A_checkworthy_multimodal_english_v2'
zip_file_path = 'data/CT23_1A_checkworthy_multimodal_english_v2.zip'

def zip_extration(folder_path, zip_file_path):
    print('Zip file extraction started')
    if not os.path.exists(folder_path):
        print('Folder does not exist, extracting zip file')
        os.makedirs(folder_path)

        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(folder_path)
    
    print('Zip file extracted')

zip_extration(folder_path, zip_file_path)

train_path = folder_path + '/CT23_1A_checkworthy_multimodal_english_train.jsonl'
test_path = folder_path + '/CT23_1A_checkworthy_multimodal_english_test.jsonl'

def split_json(data):
    text_data = {
        'tweet_id': data['tweet_id'],
        'tweet_url': data['tweet_url'],
        'tweet_text': data['tweet_text'],
        'ocr_text': data['ocr_text'],
        'class_label': data['class_label']
    }

    image_data = {
        'tweet_id': data['tweet_id'],
        'tweet_url': data['tweet_url'],
        'class_label': data['class_label'],
        'image_path': data['image_path'],
        'image_url': data['image_url']
    }

    return text_data, image_data


# Read data from the folder
def read_data(file_path):
    text_data = []
    image_data = []
    with open(file_path, 'r') as file:
        for line in file:
            json_obj = json.loads(line)
            text, image = split_json(json_obj)
            text_data.append(text)
            image_data.append(image)
    return text_data, image_data

train_text_data, train_image_data = read_data(train_path)
print(f'Text: {train_text_data[0]}')
print(f'Image: {train_image_data[0]}')


Zip file extraction started
Zip file extracted
Text: {'tweet_id': '1222845188567003136', 'tweet_url': 'https://twitter.com/user/status/1222845188567003136', 'tweet_text': 'Coronavirus: DED warns pharmacies against hiking prices of face masks https://t.co/ZBvkOKiuex https://t.co/GOiAsDGP75', 'ocr_text': 'WARNING\nM 8210\ncertain particles\nreut in eickness\nFor proper use, see\nor or box or call 3M\n0-247-3941. NIOSH\nN95\nagainst certain particies.\nheips protect\nЗи 8270\nMisuse may result in sickness\nsupervisor or box or call 3M\nor death. For proper use, see\nThis respirator helps protect\n1-800-247-3941. NIOSH\nCM0007\nLOF R17271\nAWARNING\nARNING\n3M 8210\nIC-84A 0007\nT R17271\ntin particles\nsicieness\nFapoper use, see\nbox or cel 3M\nB3C-841 NIOSH\nN95\nTC-8AU\nLOTE R17271\nwING\nN95\nM 8210\n', 'class_label': 'Yes'}
Image: {'tweet_id': '1222845188567003136', 'tweet_url': 'https://twitter.com/user/status/1222845188567003136', 'class_label': 'Yes', 'image_path': 'images_labeled

## Clean Data

In [ ]:
# Clean data here (maybe lemmatizer and such)


## Run Text Analyzer Model

In [ ]:
# Train model here
# Use RoBERTa model or GPT model from Hugging Face

## Run Image Analyser Model

In [ ]:
# Train model here
# Use VIT model from Hugging Face

## Concatenate The Two Models

In [ ]:
# Concatenate models here two get the image and text results together